# Paper 4: Recurrent Neural Network Regularization
## Wojciech Zaremba, Ilya Sutskever, Oriol Vinyals (2014)

### Dropout for RNNs

Key insight: Apply dropout to **non-recurrent connections only**, not recurrent connections.

In [ ]:
import torch
import matplotlib.pyplot as plt

torch.manual_seed(42)
# Torch-Numpy compatibility helpers
if not hasattr(torch.Tensor, 'copy'):
    torch.Tensor.copy = torch.Tensor.clone

def _astype(self, dtype):
    """
    Compute  astype.
    
    Args:
        dtype: Input parameter.
    Returns:
        Computed result.
    """
    if dtype is float:
        return self.float()
    if dtype is int:
        return self.int()
    return self.to(dtype)

if not hasattr(torch.Tensor, 'astype'):
    torch.Tensor.astype = _astype


## Standard Dropout

In [ ]:
def dropout(x, dropout_rate=0.5, training=True):
    """
    Compute dropout.
    
    Args:
        x: Input data.
        dropout_rate: Rate parameter.
        training: Input parameter.
    Returns:
        Computed result.
    """
    if not training or dropout_rate == 0:
        return x
    
    # Inverted dropout (scale during training)
    mask = (torch.rand(*x.shape) > dropout_rate).astype(float)
    return x * mask / (1 - dropout_rate)

# Test dropout
x = torch.ones((5, 1))
print("Original:", x.T)
print("With dropout (p=0.5):", dropout(x, 0.5).T)
print("With dropout (p=0.5):", dropout(x, 0.5).T)
print("Test mode:", dropout(x, 0.5, training=False).T)

## RNN with Proper Dropout

**Key**: Dropout on **inputs** and **outputs**, NOT on recurrent connections!

In [ ]:
class RNNWithDropout:
    def __init__(self, input_size, hidden_size, output_size):
        """
        Initialize the instance.
        
        Args:
            input_size: Size of input.
            hidden_size: Size of hidden.
            output_size: Size of output.
        Returns:
            Computed result.
        """
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Weights
        self.W_xh = torch.randn(hidden_size, input_size) * 0.01
        self.W_hh = torch.randn(hidden_size, hidden_size) * 0.01
        self.W_hy = torch.randn(output_size, hidden_size) * 0.01
        self.bh = torch.zeros((hidden_size, 1))
        self.by = torch.zeros((output_size, 1))
    
    def forward(self, inputs, dropout_rate=0.0, training=True):
        """
        Run the forward pass value.
        
        Args:
            inputs: Input data.
            dropout_rate: Rate parameter.
            training: Input parameter.
        Returns:
            Computed result.
        """
        h = torch.zeros((self.hidden_size, 1))
        outputs = []
        hidden_states = []
        
        for x in inputs:
            # Apply dropout to INPUT
            x_dropped = dropout(x, dropout_rate, training)
            
            # RNN update (NO dropout on recurrent connection)
            h = torch.tanh(
                torch.matmul(self.W_xh, x_dropped) +  # Dropout HERE
                torch.matmul(self.W_hh, h) +           # NO dropout HERE
                self.bh
            )
            
            # Apply dropout to HIDDEN state before output
            h_dropped = dropout(h, dropout_rate, training)
            
            # Output
            y = torch.matmul(self.W_hy, h_dropped) + self.by  # Dropout HERE
            
            outputs.append(y)
            hidden_states.append(h)
        
        return outputs, hidden_states

# Test
rnn = RNNWithDropout(input_size=10, hidden_size=20, output_size=10)
test_inputs = [torch.randn(10, 1) for _ in range(5)]

outputs_train, _ = rnn.forward(test_inputs, dropout_rate=0.5, training=True)
outputs_test, _ = rnn.forward(test_inputs, dropout_rate=0.5, training=False)

print(f"Training output[0] mean: {outputs_train[0].mean():.4f}")
print(f"Test output[0] mean: {outputs_test[0].mean():.4f}")

## Variational Dropout

**Key innovation**: Use **same** dropout mask across all timesteps!

In [ ]:
class RNNWithVariationalDropout:
    def __init__(self, input_size, hidden_size, output_size):
        """
        Initialize the instance.
        
        Args:
            input_size: Size of input.
            hidden_size: Size of hidden.
            output_size: Size of output.
        Returns:
            Computed result.
        """
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Weights (same as before)
        self.W_xh = torch.randn(hidden_size, input_size) * 0.01
        self.W_hh = torch.randn(hidden_size, hidden_size) * 0.01
        self.W_hy = torch.randn(output_size, hidden_size) * 0.01
        self.bh = torch.zeros((hidden_size, 1))
        self.by = torch.zeros((output_size, 1))
    
    def forward(self, inputs, dropout_rate=0.0, training=True):
        """
        Run the forward pass value.
        
        Args:
            inputs: Input data.
            dropout_rate: Rate parameter.
            training: Input parameter.
        Returns:
            Computed result.
        """
        h = torch.zeros((self.hidden_size, 1))
        outputs = []
        hidden_states = []
        
        # Generate masks ONCE for entire sequence
        if training and dropout_rate > 0:
            input_mask = (torch.rand(self.input_size, 1) > dropout_rate).astype(float) / (1 - dropout_rate)
            hidden_mask = (torch.rand(self.hidden_size, 1) > dropout_rate).astype(float) / (1 - dropout_rate)
        else:
            input_mask = torch.ones((self.input_size, 1))
            hidden_mask = torch.ones((self.hidden_size, 1))
        
        for x in inputs:
            # Apply SAME mask to each input
            x_dropped = x * input_mask
            
            # RNN update
            h = torch.tanh(
                torch.matmul(self.W_xh, x_dropped) +
                torch.matmul(self.W_hh, h) +
                self.bh
            )
            
            # Apply SAME mask to each hidden state
            h_dropped = h * hidden_mask
            
            # Output
            y = torch.matmul(self.W_hy, h_dropped) + self.by
            
            outputs.append(y)
            hidden_states.append(h)
        
        return outputs, hidden_states

# Test variational dropout
var_rnn = RNNWithVariationalDropout(input_size=10, hidden_size=20, output_size=10)
outputs_var, _ = var_rnn.forward(test_inputs, dropout_rate=0.5, training=True)

print("Variational dropout uses consistent masks across timesteps")

## Compare Dropout Strategies

In [ ]:
# Generate synthetic sequence data
seq_length = 20
test_sequence = [torch.randn(10, 1) for _ in range(seq_length)]

# Run with different strategies
_, h_no_dropout = rnn.forward(test_sequence, dropout_rate=0.0, training=False)
_, h_standard = rnn.forward(test_sequence, dropout_rate=0.5, training=True)
_, h_variational = var_rnn.forward(test_sequence, dropout_rate=0.5, training=True)

# Convert to arrays
h_no_dropout = torch.hstack([h.flatten() for h in h_no_dropout]).T
h_standard = torch.hstack([h.flatten() for h in h_standard]).T
h_variational = torch.hstack([h.flatten() for h in h_variational]).T

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].imshow(h_no_dropout, cmap='RdBu', aspect='auto')
axes[0].set_title('No Dropout')
axes[0].set_xlabel('Hidden Unit')
axes[0].set_ylabel('Time Step')

axes[1].imshow(h_standard, cmap='RdBu', aspect='auto')
axes[1].set_title('Standard Dropout (different masks per timestep)')
axes[1].set_xlabel('Hidden Unit')
axes[1].set_ylabel('Time Step')

axes[2].imshow(h_variational, cmap='RdBu', aspect='auto')
axes[2].set_title('Variational Dropout (same mask all timesteps)')
axes[2].set_xlabel('Hidden Unit')
axes[2].set_ylabel('Time Step')

plt.tight_layout()
plt.show()

print("Variational dropout shows consistent patterns (same units dropped throughout)")

## Dropout Placement Matters!

In [ ]:
# Visualize where dropout is applied
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Create a simple RNN diagram
def draw_rnn_cell(ax, title, show_input_dropout, show_hidden_dropout, show_recurrent_dropout):
    """
    Compute draw recurrent neural network cell.
    
    Args:
        ax: Input parameter.
        title: Input parameter.
        show_input_dropout: Input parameter.
        show_hidden_dropout: Input parameter.
        show_recurrent_dropout: Input parameter.
    Returns:
        None.
    """
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.axis('off')
    ax.set_title(title, fontsize=12, fontweight='bold')
    
    # Draw boxes
    # Input
    ax.add_patch(plt.Rectangle((1, 2), 1.5, 1, fill=True, color='lightblue', ec='black'))
    ax.text(1.75, 2.5, 'x_t', ha='center', va='center', fontsize=10)
    
    # Hidden (current)
    ax.add_patch(plt.Rectangle((4, 4.5), 2, 2, fill=True, color='lightgreen', ec='black'))
    ax.text(5, 5.5, 'h_t', ha='center', va='center', fontsize=12)
    
    # Hidden (previous)
    ax.add_patch(plt.Rectangle((7, 4.5), 2, 2, fill=True, color='lightyellow', ec='black'))
    ax.text(8, 5.5, 'h_{t-1}', ha='center', va='center', fontsize=10)
    
    # Output
    ax.add_patch(plt.Rectangle((4, 7.5), 2, 1, fill=True, color='lightcoral', ec='black'))
    ax.text(5, 8, 'y_t', ha='center', va='center', fontsize=10)
    
    # Arrows
    # Input to hidden
    color_input = 'red' if show_input_dropout else 'black'
    width_input = 3 if show_input_dropout else 1
    ax.arrow(2.5, 2.5, 1.3, 2, head_width=0.3, color=color_input, lw=width_input)
    if show_input_dropout:
        ax.text(3.2, 3.5, 'DROPOUT', fontsize=8, color='red', fontweight='bold')
    
    # Recurrent
    color_rec = 'red' if show_recurrent_dropout else 'black'
    width_rec = 3 if show_recurrent_dropout else 1
    ax.arrow(7, 5.5, -0.8, 0, head_width=0.3, color=color_rec, lw=width_rec)
    if show_recurrent_dropout:
        ax.text(6.5, 6.2, 'DROPOUT', fontsize=8, color='red', fontweight='bold')
    
    # Hidden to output
    color_hidden = 'red' if show_hidden_dropout else 'black'
    width_hidden = 3 if show_hidden_dropout else 1
    ax.arrow(5, 6.6, 0, 0.7, head_width=0.3, color=color_hidden, lw=width_hidden)
    if show_hidden_dropout:
        ax.text(5.5, 7, 'DROPOUT', fontsize=8, color='red', fontweight='bold')

# Wrong: dropout everywhere
draw_rnn_cell(axes[0, 0], 'WRONG: Dropout Everywhere\n(Disrupts temporal flow)', 
             show_input_dropout=True, show_hidden_dropout=True, show_recurrent_dropout=True)

# Wrong: only recurrent
draw_rnn_cell(axes[0, 1], 'WRONG: Only Recurrent\n(Loses gradient flow)', 
             show_input_dropout=False, show_hidden_dropout=False, show_recurrent_dropout=True)

# Correct: Zaremba et al.
draw_rnn_cell(axes[1, 0], 'CORRECT: Zaremba et al.\n(Input & Output only)', 
             show_input_dropout=True, show_hidden_dropout=True, show_recurrent_dropout=False)

# No dropout
draw_rnn_cell(axes[1, 1], 'Baseline: No Dropout\n(May overfit)', 
             show_input_dropout=False, show_hidden_dropout=False, show_recurrent_dropout=False)

plt.tight_layout()
plt.show()

## Key Takeaways

### The Problem:
- Naive dropout on RNNs doesn't work well
- Dropping recurrent connections disrupts temporal information flow
- Standard dropout changes mask every timestep (noisy)

### Zaremba et al. Solution:

**Apply dropout to:**
- ✅ Input-to-hidden connections (W_xh)
- ✅ Hidden-to-output connections (W_hy)

**Do NOT apply to:**
- ❌ Recurrent connections (W_hh)

### Variational Dropout:
- Use **same dropout mask** for all timesteps
- More stable than changing mask
- Better theoretical justification (Bayesian)

### Results:
- Significant improvement on language modeling
- Penn Treebank: Test perplexity improved from 78.4 to 68.7
- Works with LSTMs and GRUs too

### Implementation Tips:
1. Use higher dropout rates (0.5-0.7) than feedforward nets
2. Apply dropout in **both** directions for bidirectional RNNs
3. Can stack multiple LSTM layers with dropout between them
4. Variational dropout: generate mask once per sequence

### Why It Works:
- Preserves temporal dependencies (no dropout on recurrence)
- Regularizes non-temporal transformations
- Forces robustness to missing input features
- Consistent masks (variational) reduce variance